# Setup environment and load data

Dataset: https://s3.hothienlac.com/yomitoon/sales_data.csv

In [1]:
import pandas as pd
import requests
from io import StringIO

In [2]:
url = 'https://s3.hothienlac.com/yomitoon/sales_data.csv'
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Kiểm tra lỗi HTTP

data_raw = pd.read_csv(StringIO(response.text))

In [3]:
data_raw

,order_id,order_date,customer_id,customer_name,city,product,category,quantity,unit_price,payment_method
0,1001,2024-01-02,C001,Alice,New York,Laptop,Electronics,1,1200,Credit Card
1,1002,2024-01-02,C002,Bob,Los Angeles,Headphones,Electronics,2,150,PayPal
2,1003,2024-01-03,C003,Charlie,New York,Office Chair,Furniture,1,350,Credit Card
3,1004,2024-01-03,C001,Alice,New York,Mouse,Electronics,3,25,Debit Card
4,1005,2024-01-04,C004,Diana,Chicago,Desk,Furniture,1,500,Bank Transfer
5,1006,2024-01-04,C005,Eve,Chicago,Laptop,Electronics,1,1100,Credit Card
6,1007,2024-01-05,C002,Bob,Los Angeles,Monitor,Electronics,2,300,Debit Card
7,1008,2024-01-05,C003,Charlie,New York,Desk Lamp,Furniture,2,45,PayPal
8,1009,2024-01-06,C006,Frank,Miami,Tablet,Electronics,1,600,Credit Card
9,1010,2024-01-06,C001,Alice,New York,Keyboard,Electronics,1,80,Bank Transfer


# 🟡 LEVEL 4 — Analytical Aggregation (Score 4–6)

## **Q8. Average order value (AOV) per customer**

### Task

For each customer, compute:

* total spending
* number of orders
* **average order value**

### 💡 Hint

Use:

* `.groupby()`
* `.agg()`
* basic arithmetic between aggregated columns

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
* [https://pandas.pydata.org/docs/user_guide/groupby.html](https://pandas.pydata.org/docs/user_guide/groupby.html)

### 🧠 Explanation

This teaches:

* multi-metric aggregation
* **ratio metrics** (very common in dashboards)
* separating *raw data* from *business KPIs*

In [4]:
data_total = data_raw
data_total['total'] = data_raw['quantity'] * data_raw['unit_price']
data_total.groupby('customer_id').agg(
    count_orders=('order_id', 'count'),
    total_spending=('total', 'sum'),
    avg_order_value=('total', 'mean')
)

,count_orders,total_spending,avg_order_value
customer_id,,,
C001,3,1355,451.666667
C002,2,900,450.000000
C003,2,440,220.000000
C004,1,500,500.000000
C005,1,1100,1100.000000
C006,1,600,600.000000


In [5]:
data_total

,order_id,order_date,customer_id,customer_name,city,product,category,quantity,unit_price,payment_method,total
0,1001,2024-01-02,C001,Alice,New York,Laptop,Electronics,1,1200,Credit Card,1200
1,1002,2024-01-02,C002,Bob,Los Angeles,Headphones,Electronics,2,150,PayPal,300
2,1003,2024-01-03,C003,Charlie,New York,Office Chair,Furniture,1,350,Credit Card,350
3,1004,2024-01-03,C001,Alice,New York,Mouse,Electronics,3,25,Debit Card,75
4,1005,2024-01-04,C004,Diana,Chicago,Desk,Furniture,1,500,Bank Transfer,500
5,1006,2024-01-04,C005,Eve,Chicago,Laptop,Electronics,1,1100,Credit Card,1100
6,1007,2024-01-05,C002,Bob,Los Angeles,Monitor,Electronics,2,300,Debit Card,600
7,1008,2024-01-05,C003,Charlie,New York,Desk Lamp,Furniture,2,45,PayPal,90
8,1009,2024-01-06,C006,Frank,Miami,Tablet,Electronics,1,600,Credit Card,600
9,1010,2024-01-06,C001,Alice,New York,Keyboard,Electronics,1,80,Bank Transfer,80


## **Q9. Revenue contribution by category (%)**

### Task

Calculate:

* total revenue per product category
* percentage contribution of each category to total revenue

### 💡 Hint

Use:

* `.groupby()`
* `.sum()`
* `.assign()`
* division by a **global scalar**

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.div.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.div.html)

### 🧠 Explanation

You learn:

* **normalization**
* how to compare groups on the same scale
* how to prepare data for **pie charts / stacked bars**

In [6]:
data_total.groupby('category').agg(
    total_revenue=('total', 'sum')
).assign(
    revenue_contribution=lambda x: x['total_revenue'] / x['total_revenue'].sum())

,total_revenue,revenue_contribution
category,,
Electronics,3955,0.807967
Furniture,940,0.192033


# 🔵 LEVEL 5 — Distribution, Ranking & Segmentation (Score 7–8)

## **Q10. Identify top 20% customers by revenue (Pareto analysis)**

### Task

Determine:

* which customers belong to the **top 20%** by total spending

### 💡 Hint

Use:

* `.sort_values()`
* `.cumsum()`
* `.quantile()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html)

### 🧠 Explanation

This is a classic **80/20 rule** problem:

* who really drives revenue?
* foundational for **customer segmentation**

In [11]:
total_revenue = data_total.groupby('customer_id').agg(
    total_spending=('total', 'sum')
).sort_values('total_spending', ascending=False)

In [19]:
# Get 20% of total_revenue
total_revenue['total_spending'][:int(len(total_revenue) * 0.2)]

,total_spending
customer_id,
C001,1355


## **Q11. Price distribution analysis per category**

### Task

For each product category, compute:

* mean unit price
* median unit price
* standard deviation

### 💡 Hint

Use:

* `.groupby()`
* `.agg(mean=..., median=..., std=...)`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html)

### 🧠 Explanation

This builds intuition for:

* **distribution shape**
* why **median ≠ mean**
* choosing the right chart (boxplot vs bar)

In [21]:
data_raw.groupby('category').agg(
    mean_unit_price=('unit_price', 'mean'),
    median_unit_price=('unit_price', 'median'),
    std_unit_price=('unit_price', 'std')
)

,mean_unit_price,median_unit_price,std_unit_price
category,,,
Electronics,493.571429,300.0,487.209010
Furniture,298.333333,350.0,231.858434


# 🔴 LEVEL 6 — Time Series, Growth & Insight (Score 9–10)

## **Q12. Day-over-day revenue growth (%)**

### Task

Compute:

* daily revenue
* **percentage change compared to previous day**

### 💡 Hint

Use:

* `.groupby()`
* `.pct_change()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.pct_change.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.pct_change.html)

### 🧠 Explanation

This teaches:

* growth vs absolute value
* preparing data for **line charts**
* understanding volatility

In [22]:
data_raw.groupby('order_date').agg(
    daily_revenue=('total', 'sum')).pct_change()

,daily_revenue
order_date,
2024-01-02,NaN
2024-01-03,-0.716667
2024-01-04,2.764706
2024-01-05,-0.568750
2024-01-06,-0.014493


## **Q13. Rolling average of daily revenue**

### Task

Calculate:

* 3-day rolling average of daily revenue

### 💡 Hint

Use:

* `.rolling(window=3)`
* `.mean()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html)

### 🧠 Explanation

Rolling metrics are used to:

* smooth noisy data
* reveal trends
* support **time-series visualization**

In [24]:
data_raw.groupby('order_date').agg(
    daily_revenue=('total', 'sum')).rolling(window=3).mean()

,daily_revenue
order_date,
2024-01-02,NaN
2024-01-03,NaN
2024-01-04,1175.0
2024-01-05,905.0
2024-01-06,990.0


## **Q14. Detect unusually large orders (outliers)**

### Task

Flag orders where:

* `total_amount` is significantly higher than normal
  (use a statistical threshold)

### 💡 Hint

Use:

* `.mean()`
* `.std()`
* boolean conditions

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html)

### 🧠 Explanation

You are learning:

* **basic anomaly detection**
* how math supports intuition
* how analysts decide what deserves investigation

In [29]:
# Tính các giá trị thống kê cần thiết
Q1 = data_total['total'].quantile(0.25)
Q3 = data_total['total'].quantile(0.75)
IQR = Q3 - Q1

# Xác định ngưỡng outlier
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Lọc các đơn hàng có giá trị bất thường
outliers = data_total[data_total['total'] > upper_bound]

outliers

,order_id,order_date,customer_id,customer_name,city,product,category,quantity,unit_price,payment_method,total
